# Recurrency Metric for the tgbl-review dataset

This notebook explores a recurrency metric for the tgbl-review dataset based on how often interaction between pairs of source and destination occur. It splits the interactions in groups based on frequncy of recurrence - 1 day, 1 week, 1 month etc. 

### Dataset Preparation

In [6]:
from TGB.tgb.linkproppred.dataset import LinkPropPredDataset
import pandas as pd

In [7]:
name = "tgbl-review"
dataset = LinkPropPredDataset(name=name, root="./Datasets", preprocess=True)
data = dataset.full_data

data_df = pd.DataFrame([data['sources'], data['destinations'], data['timestamps']]).T
data_df.columns = ['Source', 'Destination', 'Timestamp']

raw file found, skipping download
Dataset directory is  /opt/homebrew/lib/python3.11/site-packages/tgb/./Datasets/tgbl_review
loading processed file


In [8]:
data_df

,Source,Destination,Timestamp
0,0.0,1.0,9.292320e+08
1,2.0,3.0,9.307872e+08
2,4.0,1.0,9.318240e+08
3,5.0,6.0,9.318240e+08
4,7.0,8.0,9.318240e+08
...,...,...,...
4873535,257355.0,234180.0,1.538438e+09
4873536,195919.0,301073.0,1.538525e+09
4873537,143936.0,137681.0,1.538525e+09
4873538,70003.0,249788.0,1.538611e+09


### Recurrency Metric based on Interaction Frequency

In [15]:
data_df['Timestamp'] = pd.to_datetime(data_df['Timestamp'], unit='s')

In [30]:
data_df['Year'] = data_df['Timestamp'].dt.year
most_common_year = data_df['Year'].value_counts().idxmax()

start_date = pd.Timestamp(year=most_common_year, month=1, day=1)
end_date = pd.Timestamp(year=most_common_year, month=3, day=31)
df_filtered = data_df[(data_df['Timestamp'] >= start_date) & (data_df['Timestamp'] <= end_date)]

df_filtered['YearWeek'] = df_filtered['Timestamp'].dt.to_period('W')

def check_weekly_occurrences(group):
    expected_weeks = pd.date_range(start=start_date, end=end_date, freq='W-MON').nunique()
    unique_weeks = group['YearWeek'].nunique()
    return unique_weeks == expected_weeks / 2

weekly_occurrences = df_filtered.groupby(['Source', 'Destination']).filter(check_weekly_occurrences)

summary = weekly_occurrences.drop_duplicates(subset=['Source', 'Destination'])

print(f"Year with the most records: {most_common_year}")
print(summary)

/var/folders/j9/gr3m7ly52mx4hfzpzdtzzfg40000gn/T/ipykernel_89750/1822288608.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_filtered['YearWeek'] = df_filtered['Timestamp'].dt.to_period('W')


Year with the most records: 2015
Empty DataFrame
Columns: [Source, Destination, Timestamp, Year, YearWeek]
Index: []


In [29]:
start_date = "2015-01-01"
end_date = "2015-03-31"
df_filtered = data_df[(data_df['Timestamp'] >= start_date) & (data_df['Timestamp'] <= end_date)]

df_filtered['YearMonth'] = df_filtered['Timestamp'].dt.to_period('M')

def check_monthly_occurrences(group):
    unique_months = group['YearMonth'].nunique()
    return unique_months == 2

monthly_occurrences = df_filtered.groupby(['Source', 'Destination']).filter(check_monthly_occurrences)

summary = monthly_occurrences.drop_duplicates(subset=['Source', 'Destination'])

print(summary)


/var/folders/j9/gr3m7ly52mx4hfzpzdtzzfg40000gn/T/ipykernel_89750/3120860158.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_filtered['YearMonth'] = df_filtered['Timestamp'].dt.to_period('M')


           Source  Destination  Timestamp  Year YearMonth
1919193  133197.0     150489.0 2015-01-05  2015   2015-01
1922440  269567.0     158789.0 2015-01-06  2015   2015-01
1924553  194963.0     149123.0 2015-01-07  2015   2015-01
1925655   13179.0     121031.0 2015-01-07  2015   2015-01
1925934  179066.0     242150.0 2015-01-07  2015   2015-01
...           ...          ...        ...   ...       ...
2081909  213310.0     222245.0 2015-02-25  2015   2015-02
2082537  312258.0     244894.0 2015-02-25  2015   2015-02
2084530   83401.0      79841.0 2015-02-26  2015   2015-02
2086926   53510.0     240722.0 2015-02-26  2015   2015-02
2091793  118566.0     172336.0 2015-02-28  2015   2015-02

[77 rows x 5 columns]


In [21]:
frequency = data_df.groupby(['Source', 'Destination'])['Timestamp'].transform(classify_frequency)
data_df['Frequency'] = frequency


In [22]:
data_df

,Source,Destination,Timestamp,Frequency
0,0.0,1.0,1999-06-13,Single occurrence
1,2.0,3.0,1999-07-01,Single occurrence
2,4.0,1.0,1999-07-13,Single occurrence
3,5.0,6.0,1999-07-13,Single occurrence
4,7.0,8.0,1999-07-13,Single occurrence
...,...,...,...,...
4873535,257355.0,234180.0,2018-10-02,Single occurrence
4873536,195919.0,301073.0,2018-10-03,Single occurrence
4873537,143936.0,137681.0,2018-10-03,Single occurrence
4873538,70003.0,249788.0,2018-10-04,Single occurrence


Now obtain only pairs that have daily interactions

In [23]:
daily_occurrences = data_df[data_df['Frequency'] == 'Daily']

In [24]:
daily_occurrences

,Source,Destination,Timestamp,Frequency
587,669.0,185.0,2000-08-01,Daily
588,669.0,100.0,2000-08-01,Daily
589,669.0,185.0,2000-08-01,Daily
590,669.0,100.0,2000-08-01,Daily
664,669.0,560.0,2000-08-17,Daily
...,...,...,...,...
4871920,154555.0,308688.0,2018-09-11,Daily
4871922,154555.0,308688.0,2018-09-11,Daily
4871923,154555.0,308688.0,2018-09-11,Daily
4871924,154555.0,308688.0,2018-09-11,Daily


In [25]:
daily_occurrences.to_csv('datasets/tgbl-review_daily_occurrences.csv', index=False)
